# **План-контент практичного заняття**  
## **Тема 3. Заняття 5. Практичний аналіз часових рядів**  
### **1. Аналіз часових рядів засобами машинного навчання**  
### **2. Аналіз часових рядів засобами глибокого навчання**  

---

## **1. Вступ (10 хв)**  

### **1.1. Мета заняття**  
- Ознайомити слухачів із методами аналізу часових рядів засобами машинного та глибокого навчання.  
- Навчити застосовувати алгоритми для прогнозування майбутніх значень часових рядів.  
- Виконати практичний аналіз часових рядів у Python, використовуючи традиційні методи машинного навчання та нейромережі (LSTM).  

### **1.2. Очікувані результати**  
Після заняття слухачі зможуть:  
✅ Використовувати моделі машинного навчання (Random Forest, XGBoost) для прогнозування часових рядів.  
✅ Виконувати аналіз часових рядів за допомогою глибокого навчання (LSTM).  
✅ Оцінювати ефективність моделей прогнозування.  

---

## **2. Аналіз часових рядів засобами машинного навчання (40 хв)**  

### **2.1. Підготовка даних для машинного навчання**  
📌 **Кроки підготовки:**  
1️⃣ Заповнення пропущених значень.  
2️⃣ Перетворення дат у числовий формат.  
3️⃣ Створення ознак (lag-функції, ковзні середні).  

📌 **Приклад коду для підготовки даних**  
```python
import pandas as pd

# Завантаження даних
df = pd.read_csv("combat_reports.csv", parse_dates=["date"])
df.set_index("date", inplace=True)

# Створення лагових змінних (значення попередніх днів)
df["lag_1"] = df["num_casualties"].shift(1)
df["lag_2"] = df["num_casualties"].shift(2)

# Заповнення пропущених значень
df.dropna(inplace=True)

print(df.head())
```

---

### **2.2. Прогнозування часових рядів за допомогою Random Forest**  
📌 **Random Forest добре працює на часових рядах, якщо використовувати ознаки lag-функції.**  

📌 **Приклад прогнозування**  
```python
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Формування вибірки
X = df[["lag_1", "lag_2"]]
y = df["num_casualties"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Навчання моделі
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Прогноз
y_pred = model.predict(X_test)

# Оцінка моделі
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")
```

📌 **Очікуваний результат:**  
```
Mean Squared Error: 5.64
```

---

### **2.3. Використання XGBoost для прогнозування**  
📌 **XGBoost – потужний алгоритм, що добре працює з часовими рядами.**  

📌 **Приклад прогнозування**  
```python
import xgboost as xgb

# Створення моделі
model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100)

# Навчання
model.fit(X_train, y_train)

# Прогноз
y_pred = model.predict(X_test)

# Оцінка моделі
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (XGBoost): {mse:.2f}")
```

📌 **Очікуваний результат:**  
```
Mean Squared Error (XGBoost): 4.78
```

---

## **3. Аналіз часових рядів засобами глибокого навчання (50 хв)**  

### **3.1. Основи використання LSTM для часових рядів**  
📌 **LSTM (Long Short-Term Memory) – вид рекурентних нейронних мереж, що добре працює з часовими рядами.**  

🔹 **Переваги LSTM:**  
✔ Враховує довгострокові залежності у даних.  
✔ Добре працює з нелінійними часовими рядами.  
✔ Використовується для прогнозування фінансових, економічних та військових часових рядів.  

---

### **3.2. Підготовка даних для LSTM**  
📌 **Перетворення даних у формат, придатний для нейронної мережі.**  

```python
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Масштабування даних
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[["num_casualties"]])

# Формування набору для навчання
X_train, y_train = [], []
for i in range(10, len(df_scaled)):
    X_train.append(df_scaled[i-10:i])
    y_train.append(df_scaled[i])

X_train, y_train = np.array(X_train), np.array(y_train)
```

---

### **3.3. Побудова LSTM-моделі**  
📌 **Створюємо LSTM-архітектуру для прогнозування часових рядів.**  

```python
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Побудова LSTM-моделі
model = Sequential([
    LSTM(50, activation="relu", return_sequences=True, input_shape=(10, 1)),
    LSTM(50, activation="relu"),
    Dense(1)
])

# Компільовуємо модель
model.compile(optimizer="adam", loss="mse")

# Навчання моделі
model.fit(X_train, y_train, epochs=20, batch_size=16)
```

📌 **Очікуваний результат:**  
```
Epoch 20/20
loss: 0.0021
```

---

### **3.4. Прогнозування майбутніх значень**  
📌 **Отримуємо прогноз на основі останніх 10 значень ряду.**  

```python
# Формуємо вхідні дані для прогнозу
X_input = df_scaled[-10:].reshape(1, 10, 1)

# Прогноз
y_pred = model.predict(X_input)

# Перетворення прогнозу у вихідний масштаб
y_pred = scaler.inverse_transform(y_pred)

print(f"Прогнозована кількість атак: {y_pred[0][0]:.2f}")
```

📌 **Очікуваний результат:**  
```
Прогнозована кількість атак: 17.34
```

---

## **4. Практичні завдання (30 хв)**  
✅ **Завдання 1:** Підготувати часовий ряд для аналізу (очистити, нормалізувати).  
✅ **Завдання 2:** Використати XGBoost для прогнозування наступного місяця.  
✅ **Завдання 3:** Побудувати LSTM-модель та виконати прогнозування.  

---

## **5. Висновки та підсумки заняття (10 хв)**  
✅ **Основні висновки:**  
- **Random Forest, XGBoost** добре працюють із часовими рядами, якщо використовувати lag-ознаки.  
- **LSTM** ефективно працює з нелінійними часовими рядами та довгостроковими залежностями.  
- **Підготовка даних (нормалізація, лаги, ковзні середні)** є критично важливою для коректного прогнозування.  

🚀 **Ключовий меседж:**  
**Поєднання традиційного машинного навчання та глибоких нейромереж дозволяє ефективно прогнозувати військові часові ряди та приймати стратегічні рішення.**